<a href="https://colab.research.google.com/github/sean-halpin/cycle_gan_aif1_mouth_only/blob/cyclegan_from_serialized_model/cyclegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Dataset


In [ ]:
import urllib.request
url = "https://inc.ucsd.edu/mplab/databases/GENKI-R2009a.zip"
file_name = "GENKI.zip"
urllib.request.urlretrieve(url, file_name)


import io, zipfile
archive = zipfile.ZipFile('GENKI.zip', 'r')
archive.extractall("dataset/")

import pandas as pd 
data = pd.read_csv(
    "dataset/GENKI-R2009a/Subsets/GENKI-4K/GENKI-4K_Labels.txt", 
    sep=" ", 
    header=None, 
    names=["smile", "pitch","yaw","roll"],
    index_col=None
)

image_symlink_location = "dataset/GENKI-R2009a/Subsets/GENKI-4K/files/"

df1 = data
df1['index_col'] = df1.index
df1['image'] = df1.apply(lambda x: image_symlink_location + "file" + str(int(x['index_col'] + 1)).zfill(4) + ".jpg", axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

In [ ]:
from matplotlib import pyplot as plt
import cv2

smiles = df1.loc[df1['smile'] == 1]
neutral = df1.loc[df1['smile'] == 0]

# Import Libraries for running model

In [ ]:
!pip install tensorflow_addons

In [ ]:
# Imports
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
print(tf.__version__)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True

session = InteractiveSession(config=config)

In [ ]:
image_x = 128
image_y = 64
IMAGE_SIZE = [image_y, image_x]


# Helper Functions

In [ ]:
def printSmileFaces(epoch=0,num=5):
  _, ax = plt.subplots(num, 2, figsize=(12, 16))
  for i, img in enumerate(neutral_ds.take(num)):
      prediction = smile_generator(img, training=False)[0].numpy()
      prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
      img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

      ax[i, 0].imshow(img)
      ax[i, 1].imshow(prediction)
      ax[i, 0].set_title("Input Photo")
      ax[i, 1].set_title("Generated Smile Photo")
      ax[i, 0].axis("off")
      ax[i, 1].axis("off")
  plt.show()
  

In [ ]:
def printNeutralFaces(epoch=0, num=5):
  _, ax = plt.subplots(num, 2, figsize=(12, 16))
  for i, img in enumerate(smile_ds.take(num)):
      prediction = neutral_generator(img, training=False)[0].numpy()
      prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
      img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

      ax[i, 0].imshow(img)
      ax[i, 1].imshow(prediction)
      ax[i, 0].set_title("Input Photo")
      ax[i, 1].set_title("Generated Nuetral Photo")
      ax[i, 0].axis("off")
      ax[i, 1].axis("off")
  plt.show()
  

In [ ]:
!mkdir -p sample_ouputs_smiling
!mkdir -p sample_ouputs_neutral

In [ ]:
from PIL import Image
def predict(model, input):
    input = tf.reshape(tf.keras.preprocessing.image.img_to_array(input), [*IMAGE_SIZE, 3])
    input = (tf.cast(input, tf.float32) / 127.5) - 1
    input = tf.expand_dims(input,axis=0)
    prediction = model(input, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    return prediction

def run_model(model, out_dir, image_path,ind_col,iterations=1,save_plot=False):
  img = cv2.imread(image_path)
  rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  plt.subplot(121)
  plt.imshow(rgb_img)
  plt.title("Input")
  # Load the face detect haar cascade
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
  # Detect faces
  faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
  # Draw rectangle around the faces and crop the faces
  found_face_already = False
  for (x, y, w, h) in faces:
    if found_face_already:
      break
    found_face_already = True
    mouth = rgb_img[y + int(h/2):y + h, x:x + w]
    orig_y,orig_x,d = mouth.shape
    resized_mouth = Image.fromarray(mouth).resize((image_x,image_y))
    # predict smile
    inp = resized_mouth
    for it in range(iterations):
      prediction=predict(model, inp)
      inp = prediction
    # Resize to original 
    resized_prediction = Image.fromarray(prediction).resize((orig_x,orig_y))
    # Add generated mouth back to original image
    gen_mouth = np.array(resized_prediction)
    overlay=cv2.addWeighted(rgb_img[y + int(h/2):y + h, x:x + w],0.25,gen_mouth,0.75,0)
    rgb_img[y + int(h/2):y + h, x:x + w] = overlay
    # Denoise
    denoise = cv2.fastNlMeansDenoisingColored(rgb_img,None,3,3,11,29)
    plt.subplot(122)
    plt.imshow(denoise)
    plt.title("Generated")
    plt.savefig(out_dir + "/input_output_"+str(ind_col)+".png")
    plt.show()

def make_smile(skip, take, save_plot):
  for i,row in neutral[skip:skip+take].iterrows():
      run_model(smile_generator, "sample_ouputs_smiling", row.image,row.index_col,1, save_plot)

def make_neutral(skip, take, save_plot):
  for i,row in smiles[skip:skip+take].iterrows():
      run_model(neutral_generator, "sample_ouputs_neutral", row.image,row.index_col,1, save_plot)

# Load Model from File

There is a prerequisite here that you have run trained the model and saved a model to your google drive. 

Train the model using the main or data_augmentation branch from the repo: https://github.com/sean-halpin/cycle_gan_aif1_mouth_only

In [ ]:
from keras.models import load_model
smile_generator = load_model("drive/MyDrive/smile_04_14_2021_18_12_06.h5")
neutral_generator = load_model("drive/MyDrive/neutral_04_14_2021_18_12_06.h5")

# Generate Model Output

In [ ]:
# First param is an index into the dataset, second param is how many faces to generate smile predictions for. 
make_smile(skip=1000,take=1,save_plot=True)

In [ ]:
import shutil
shutil.make_archive("sample_output_smiles", 'zip', "sample_ouputs_smiling")

In [ ]:
make_neutral(skip=1000,take=1,save_plot=True)

In [ ]:
shutil.make_archive("sample_output_neutrals", 'zip', "sample_ouputs_neutral")

# Demonstration with Upload

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
for f in uploaded: 
  print(f)
  run_smile_model(f,1)